# Analyzing.

## 1. Location.

In [137]:
from pymongo import MongoClient
import pandas as pd
import time
import importlib
import geopandas as gpd
import data.cleaning as clean
importlib.reload(clean)
import json
from cartoframes.viz import Map, Layer, popup_element

Todo el proceso para descargar la base de datos Empresas de Mongo.

In [138]:
client = MongoClient("localhost:27017")

In [139]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [140]:
client.list_database_names()

['Ironhack', 'admin', 'config', 'local']

In [141]:
db = client["Ironhack"]

In [142]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack')

In [143]:
c = db.get_collection("companies")

Explore the data base to get the information I want for the company.

In [144]:
# This was to look if the data base from Mongo was well downloaded.
# c.find_one()

In [145]:
# This was to find with a specific condition of the ategory, and look for what i wanted to work on.
# list(c.find({}, {"category_code":1}))

#### Will choose the music industry for my company.
Getting the music buissnes companies and start analizing them.

In [146]:
# list of all the companies with category of music in all the world.
#list(c.find({"category_code":"music"}, {"name":1, "_id":0, "category_code":1}))

Get the companies Ubication COUNTRY.

In [147]:
def music_countries (category_code, country_code):
    condition1 = {"category_code":category_code}
    projection = {"name":1, "_id":0, "offices.country_code":country_code}
    countries = (list(c.find(condition1, 
               projection)))
    
    listcountry = [countries[i]["offices"][j]["country_code"] for i in range(len(countries)) for j in range(len(countries[i]["offices"]))]
    return listcountry
    

In [148]:
# list of the countries with the music company industry.
lstcountry = music_countries ("music", 1)

In [149]:
#countries[0]
#countries[0]["offices"]
#countries[0]["offices"][0]["country_code"]

In [150]:
# change it to a dictionary
country = {"country": lstcountry}

In [151]:
# make it a data frame
dfcountry = pd.DataFrame(country)

We see that for much USA is the place for a music company, I wanted to see if I could go to another country that had some type of competition between the USA, but if the company wants to grow, need to be in the USA.

In [152]:
dfcountry.value_counts()[:10]

country
USA        30
GBR         5
FRA         4
DEU         2
BEL         1
BRA         1
ESP         1
ITA         1
LUX         1
NLD         1
dtype: int64

hacer otra columna del value count de los paises

In [153]:
dfcountry.sample(3)

,country
2,USA
16,GBR
21,FRA


### Searching for the city

In [154]:
condition1 = {"category_code":"music"}
condition2 = {"offices.city":"San Diego"}
projection = {"name":1, "_id":0, "offices.address1":1}
x = (list(c.find({"$and":
                [condition1, condition2]},
                   projection)))

  

In [155]:
x

[{'name': 'Slacker',
  'offices': [{'address1': '16935 W. Bernardo Dr. Suite 101'}]},
 {'name': 'Bandsintown',
  'offices': [{'address1': '800 W Ivy Street, Unit C'},
   {'address1': '215 Lexington Ave'}]}]

In [156]:
def music_cities (category_code, cities):
    condition1 = {"category_code":category_code}
    projection = {"name":1, "_id":0, "offices.city":cities}
    city = (list(c.find(condition1, 
               projection)))
    
    lstcity = [city[i]["offices"][j]["city"] for i in range(len(city)) for j in range(len(city[i]["offices"]))]
    return lstcity

In [157]:
lstcity = music_cities ("music", 1)

Merge the 2 columns of country and city and new count column

In [158]:
def combine_lists (lst1, lst2):
    # change the 2 lists into a dictionary
    location = {"cities": lst1,
            "country": lst2} 
    # create them a Data Frame.
    location = pd.DataFrame(location)
    # create a new column with the value_counts of cities.
    location["count"] = location.groupby(["cities"])["country"].transform("count")
    return location

In [159]:
location_df = combine_lists (lstcity, lstcountry)

In [160]:
location_df.sample(3)

,cities,country,count
29,Framingham,USA,1
16,London,GBR,4
27,San Francisco,USA,9


Will do a drop duplicated and drop na 

In [161]:
def clean_df_citycountry (df):
    x = df.combine_first(df).reset_index().reindex(df.columns, axis=1).drop_duplicates()
    x.dropna(how="any", inplace=True)
    return x

In [162]:
location_clean = clean_df_citycountry (location_df)
location_clean.sample(3)

,cities,country,count
0,San Diego,USA,2
32,Seattle,USA,1
29,Framingham,USA,1


Export the data frame of location into a CSV file.

In [163]:
def export_location (df):
    return df.to_csv(f"D:\\ironhack\\proyectos\\GeoSpatialData_proy3\csv\\location_clean.csv")
    

In [164]:
export_location (location_clean)

## 2. Startup environment.

In [165]:
def startups_cooord (city, amount):
    # Conditions for city and amount of the company.
    condition1 = {"offices.city":city}
    condition2 = {"acquisitions.price_amount":{"$gte":amount}}
    projection = {"name":1, "_id":0, "category_code":1, "offices.latitude":1, "offices.longitude":1}
    x = (list(c.find({"$and":
                             [condition1, condition2]},
                   projection)))
    return x

In [166]:
len(startups)

40

In [167]:
startups = startupsdf ("New York", 1000000)
startups

[{'name': 'Facebook',
  'category_code': 'social',
  'offices': [{'latitude': 37.41605, 'longitude': -122.151801},
   {'latitude': 53.344104, 'longitude': -6.267494},
   {'latitude': 40.7557162, 'longitude': -73.9792469}]},
 {'name': 'Viacom',
  'category_code': 'web',
  'offices': [{'latitude': 40.757725, 'longitude': -73.986011}]},
 {'name': 'Google',
  'category_code': 'search',
  'offices': [{'latitude': 37.421972, 'longitude': -122.084143},
   {'latitude': 42.280988, 'longitude': -83.748882},
   {'latitude': 33.781466, 'longitude': -84.387519},
   {'latitude': 30.351416, 'longitude': -97.751382},
   {'latitude': 40.021416, 'longitude': -105.260931},
   {'latitude': 42.36305, 'longitude': -71.087478},
   {'latitude': 41.889474, 'longitude': -87.628912},
   {'latitude': 32.981465, 'longitude': -97.018196},
   {'latitude': 32.809161, 'longitude': -96.807693},
   {'latitude': 39.62919, 'longitude': -104.898419},
   {'latitude': 42.547807, 'longitude': -83.215498},
   {'latitude': 33.6

In [168]:
def lat_startup (dict):
    x = []
    for i in range(len(startups)):
        x.append(startups[i]["offices"][0]["latitude"])
    return x

In [169]:
def long_startup (dict):
    y = []
    for i in range(len(startups)):
        y.append(startups[i]["offices"][0]["longitude"])
    return y

In [170]:
startups_lat = lat_startup (startups)

In [171]:
startups_long = long_startup (startups)

In [172]:
# Filter again the startups but without the latitud and logitude and turn it into a data frame.
def startups_name (city, amount):
    # Conditions for city and amount of the company.
    condition1 = {"offices.city":city}
    condition2 = {"acquisitions.price_amount":{"$gte":amount}}
    projection = {"name":1, "_id":0, "category_code":1}
    x = (list(c.find({"$and":
                             [condition1, condition2]},
                   projection)))
    # give a Data frame with conditions and address.
    df = pd.DataFrame(x)
    return df    

In [173]:
start_names = startups_name ("New York", 1000000)
start_names.sample(3)

,name,category_code
31,Pfizer,biotech
33,Internap,web
16,MTV Networks,advertising


Make a dictionary with all values to create a data base.

In [174]:
# dictionary for latitud and long, to create df.
def coord_df (lats, longs):
    lon_lat = {"latitud": lats,
              "longitude": longs}
    df = pd.DataFrame(lon_lat)
    return df

In [175]:
df = coord_df (startups_lat, startups_long)

In [176]:
df

,latitud,longitude
0,37.416050,-122.151801
1,40.757725,-73.986011
2,37.421972,-122.084143
3,42.362993,-71.086596
4,40.745172,-74.007646
5,40.731132,-73.991931
6,40.751816,-73.971655
7,40.731118,-73.991933
8,40.733276,-73.993279
9,39.044694,-94.597896


In [177]:
# concat the 2 df.
def concat_axis1 (df1, df2):
    starts = pd.concat([df1, df2], axis=1)
    return starts

In [182]:
startups = concat_axis1 (start_names, df)

In [189]:
startups.sample()

,name,category_code,latitud,longitude,geometry
38,Backstage,web,NaN,NaN,POINT EMPTY


Get to de df new points columns for plot it in the map.

Export clean Data frame of startups, their category and their address inside San diego.
export dictionary for maping.

In [130]:
with open ("startups.json", "w") as f:
    json.dump(startups, f)

TypeError: Object of type DataFrame is not JSON serializable

In [187]:
def export_NY_startups (df):
    return df.to_csv(f"D:\\ironhack\\proyectos\\GeoSpatialData_proy3\csv\\startups_newyork.csv")

In [188]:
export_NY_startups (gdf)

In [ ]:
Map(Layer(stores_gdf, size_continuous_style('revenue', size_range=[10,40]), title='Annual Revenue ($)'))